<a href="https://colab.research.google.com/github/MarinaCRezende/PUCRJ-AD-MVP/blob/main/MVP_MarinaRezende_AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏎️ Análise Histórica da Fórmula 1 (1950–2024)

Nome: Marina Rezende\
Sprint: Análise de Dados e Boas Práticas\
Curso: Ciência de Dados e Analytics

Este trabalho apresenta uma análise exploratória e de pré-processamento dos dados da Fórmula 1 desde sua criação, com foco em pilotos, construtores e eventos marcantes ao longo das décadas.

## Descrição do Problema

Este é um problema de **análise exploratória de dados**. O objetivo do trabalho é investigar os padrões históricos de desempenho de pilotos e equipes na Fórmula 1 entre 1950 e 2024.  

Através da análise de dados históricos, buscou-se responder perguntas como:
- Quais pilotos dominaram cada década?
- Qual a evolução da participação das equipes e pilotos ao longo dos anos?
- Quais pilotos acumularam mais voltas mais rápidas?
- Há alguma relação visível entre eventos históricos (como mudanças de regulamento) e alterações nos padrões de performance?

Essa análise poderá gerar insights relevantes sobre a evolução do esporte ao longo das décadas.


## Dicionário de Dados

Fonte de dados: **Formula 1 World Championship (1950 - 2024)** \
Acesso em <https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020/data>

\
### Datasets principais

| Dataset                     | Uso principal                                  |
| --------------------------- | ---------------------------------------------- |
| `drivers.csv`               | Informações dos pilotos                        |
| `constructors.csv`          | Informações das equipes                        |
| `races.csv`                 | Datas e locais das corridas                    |
| `results.csv`               | Resultados das corridas (posição, pontos etc.) |
| `driver_standings.csv`      | Pontuação acumulada dos pilotos                |
| `constructor_standings.csv` | Pontuação acumulada dos construtores           |
| `qualifying.csv`            | Tempo das voltas na qualificação               |
| `lap_times.csv`             | Duração das voltas nas corridas                |

\\
### Principais atributos


| Dataset                     | Atributos principais utilizados                                                            |
| --------------------------- | ------------------------------------------------------------------------------------------ |
| `drivers.csv`               | `driverId`, `dob`, `surname`, `nationality`                                                |
| `constructors.csv`          | `constructorId`, `name`                                                                    |
| `races.csv`                 | `raceId`, `year`, `circuitId`, `name`, `date`                                              |
| `results.csv`               | `raceId`, `driverId`, `constructorId`, `positionOrder`, `fastestLap`                       |
| `driver_standings.csv`      | `raceId`, `driverId`, `points`, `position`                                                 |
| `constructor_standings.csv` | `raceId`, `constructorId`, `points`, `position`                                            |
| `qualifying.csv`            | `raceId`, `driverId`, `q1`, `q2`, `q3` (convertidos em segundos com `convert_time_to_sec`) |
| `lap_times.csv`             | `raceId`, `driverId`, `lap`, `milliseconds` (convertidos em `lap_sec`)                     |
| `circuits.csv`              | `circuitId`, `name`, `location`, `country`                                                 |



## Importar bibliotecas

In [1]:
!pip install plotly

In [2]:
# Bibliotecas principais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interact, widgets


# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings('ignore')


## Carregamento dos Dados

Carregamento dos principais arquivos da base:

In [3]:
# URLs dos arquivos CSV no GitHub
url_base = "https://raw.githubusercontent.com/MarinaCRezende/PUCRJ-AD-MVP/Datasets/"

drivers_url = url_base + "drivers.csv"
constructors_url = url_base + "constructors.csv"
races_url = url_base + "races.csv"
results_url = url_base + "results.csv"
driver_standings_url = url_base + "driver_standings.csv"
constructor_standings_url = url_base + "constructor_standings.csv"
status_url = url_base + "status.csv"
circuits_url = url_base + "circuits.csv"
lap_times_url = url_base + "lap_times.csv"
qualifying_url = url_base + "qualifying.csv"

# Função para carregar arquivos CSV com verificações adicionais
def load_csv(url):
    try:
        return pd.read_csv(url, delimiter=',', encoding='utf-8')
    except pd.errors.ParserError:
        print(f"Erro ao carregar o arquivo: {url}")
        return None

# Carregar os arquivos CSV diretamente do GitHub
drivers = load_csv(drivers_url)
constructors = load_csv(constructors_url)
races = load_csv(races_url)
results = load_csv(results_url)
driver_standings = load_csv(driver_standings_url)
constructor_standings = load_csv(constructor_standings_url)
status = load_csv(status_url)
circuits = load_csv(circuits_url)
lap_times = load_csv(lap_times_url)
qualifying = load_csv(qualifying_url)

# Verificar se os dados foram carregados corretamente
print(drivers.head())
print(constructors.head())
print(races.head())
print(results.head())
print(driver_standings.head())
print(constructor_standings.head())
print(status.head())
print(circuits.head())
print(lap_times.head())
print(qualifying.head())

   driverId   driverRef number code  forename     surname         dob  \
0         1    hamilton     44  HAM     Lewis    Hamilton  1985-01-07   
1         2    heidfeld     \N  HEI      Nick    Heidfeld  1977-05-10   
2         3     rosberg      6  ROS      Nico     Rosberg  1985-06-27   
3         4      alonso     14  ALO  Fernando      Alonso  1981-07-29   
4         5  kovalainen     \N  KOV    Heikki  Kovalainen  1981-10-19   

  nationality                                             url  
0     British     http://en.wikipedia.org/wiki/Lewis_Hamilton  
1      German      http://en.wikipedia.org/wiki/Nick_Heidfeld  
2      German       http://en.wikipedia.org/wiki/Nico_Rosberg  
3     Spanish    http://en.wikipedia.org/wiki/Fernando_Alonso  
4     Finnish  http://en.wikipedia.org/wiki/Heikki_Kovalainen  
   constructorId constructorRef        name nationality  \
0              1        mclaren     McLaren     British   
1              2     bmw_sauber  BMW Sauber      German   


## Análise Exploratória

In [4]:
# Informações iniciais dos principais dataframes
print(drivers.info())
print(constructors.info())
print(results.info())
print(races.info())
print(driver_standings.info())
print(constructor_standings.info())
print(status.info())
print(circuits.info())
print(lap_times.info())
print(qualifying.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861 entries, 0 to 860
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   driverId     861 non-null    int64 
 1   driverRef    861 non-null    object
 2   number       861 non-null    object
 3   code         861 non-null    object
 4   forename     861 non-null    object
 5   surname      861 non-null    object
 6   dob          861 non-null    object
 7   nationality  861 non-null    object
 8   url          861 non-null    object
dtypes: int64(1), object(8)
memory usage: 60.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   constructorId   212 non-null    int64 
 1   constructorRef  212 non-null    object
 2   name            212 non-null    object
 3   nationality     212 non-null    object
 4   url             

In [5]:
# Exibir estatísticas descritivas de todas as planilhas
display(drivers.describe())
display(constructors.describe())
display(results.describe())
display(races.describe())
display(driver_standings.describe())
display(constructor_standings.describe())
display(status.describe())
display(circuits.describe())
display(lap_times.describe())
display(qualifying.describe())

,driverId
count,861.000000
mean,431.061556
std,248.793797
min,1.000000
25%,216.000000
50%,431.000000
75%,646.000000
max,862.000000


,constructorId
count,212.000000
mean,107.547170
std,61.952685
min,1.000000
25%,54.750000
50%,107.500000
75%,160.250000
max,215.000000


,resultId,raceId,driverId,constructorId,grid,positionOrder,points,laps,statusId
count,26759.000000,26759.000000,26759.000000,26759.000000,26759.000000,26759.000000,26759.000000,26759.000000,26759.000000
mean,13380.977391,551.687283,278.673530,50.180537,11.134796,12.794051,1.987632,46.301768,17.224971
std,7726.134642,313.265036,282.703039,61.551498,7.202860,7.665951,4.351209,29.496557,26.026104
min,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
25%,6690.500000,300.000000,57.000000,6.000000,5.000000,6.000000,0.000000,23.000000,1.000000
50%,13380.000000,531.000000,172.000000,25.000000,11.000000,12.000000,0.000000,53.000000,10.000000
75%,20069.500000,811.000000,399.500000,63.000000,17.000000,18.000000,2.000000,66.000000,14.000000
max,26764.000000,1144.000000,862.000000,215.000000,34.000000,39.000000,50.000000,200.000000,141.000000


,raceId,year,round,circuitId
count,1125.000000,1125.000000,1125.000000,1125.000000
mean,565.710222,1992.703111,8.579556,23.889778
std,328.813817,20.603848,5.159910,19.633527
min,1.000000,1950.000000,1.000000,1.000000
25%,282.000000,1977.000000,4.000000,9.000000
50%,563.000000,1994.000000,8.000000,18.000000
75%,845.000000,2011.000000,13.000000,34.000000
max,1144.000000,2024.000000,24.000000,80.000000


,driverStandingsId,raceId,driverId,points,position,wins
count,34863.000000,34863.000000,34863.000000,34863.000000,34863.000000,34863.000000
mean,43176.154232,584.413562,316.932909,14.704423,19.716720,0.277343
std,21934.276898,292.275820,274.665660,38.978094,16.293401,1.032618
min,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000
25%,19834.500000,354.000000,88.000000,0.000000,8.000000,0.000000
50%,50044.000000,603.000000,223.000000,1.000000,16.000000,0.000000
75%,62054.500000,805.000000,521.000000,10.000000,26.000000,0.000000
max,73270.000000,1144.000000,862.000000,575.000000,108.000000,19.000000


,constructorStandingsId,raceId,constructorId,points,position,wins
count,13391.000000,13391.000000,13391.000000,13391.000000,13391.000000,13391.000000
mean,16982.110821,535.399821,49.603390,37.178515,7.226047,0.697409
std,8868.446172,307.705928,61.213953,84.346048,4.355923,1.879569
min,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000
25%,8883.500000,302.000000,6.000000,0.000000,4.000000,0.000000
50%,20634.000000,508.000000,25.000000,7.000000,7.000000,0.000000
75%,25047.500000,740.000000,58.000000,33.000000,10.000000,0.000000
max,28982.000000,1144.000000,215.000000,860.000000,22.000000,21.000000


,statusId
count,139.000000
mean,71.237410
std,41.092434
min,1.000000
25%,35.500000
50%,72.000000
75%,106.500000
max,141.000000


,circuitId,lat,lng,alt
count,77.000000,77.000000,77.000000,77.000000
mean,39.883117,33.442925,1.076683,247.012987
std,23.001701,22.808866,65.516951,362.738469
min,1.000000,-37.849700,-118.189000,-7.000000
25%,20.000000,32.777400,-9.394170,18.000000
50%,40.000000,40.951700,3.930830,129.000000
75%,59.000000,46.958900,19.248600,332.000000
max,80.000000,57.265300,144.968000,2227.000000


,raceId,driverId,lap,position,milliseconds
count,589081.000000,589081.000000,589081.000000,589081.000000,5.890810e+05
mean,600.544465,325.796446,30.018104,9.661951,9.579945e+04
std,434.375976,387.561736,18.407126,5.528553,7.639973e+04
min,1.000000,1.000000,1.000000,1.000000,5.540400e+04
25%,140.000000,16.000000,14.000000,5.000000,8.204100e+04
50%,861.000000,48.000000,29.000000,9.000000,9.060800e+04
75%,1003.000000,822.000000,44.000000,14.000000,1.019300e+05
max,1144.000000,862.000000,87.000000,24.000000,7.507547e+06


,qualifyId,raceId,driverId,constructorId,number,position
count,10494.000000,10494.000000,10494.000000,10494.000000,10494.000000,10494.000000
mean,5262.482847,624.600915,343.002287,47.918430,18.786449,11.195826
std,3046.588486,428.298147,389.586448,73.217993,18.447502,6.260374
min,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
25%,2625.250000,160.000000,17.000000,4.000000,7.000000,6.000000
50%,5249.500000,870.000000,60.000000,9.000000,14.000000,11.000000
75%,7893.750000,1006.000000,822.000000,31.000000,22.000000,16.000000
max,10551.000000,1144.000000,862.000000,215.000000,99.000000,28.000000


In [6]:
# Paleta baseada na Fórmula 1 (cores comuns de equipes e F1 oficial)
f1_colors = ['#e10600',  # Vermelho Ferrari / F1 logo
             '#1e41ff',  # Azul Red Bull / Alpine
             '#00d2be',  # Verde água Mercedes
             '#f5f5f5',  # Branco Haas / McLaren 80s
             '#ff8700',  # Laranja McLaren
             '#003399',  # Azul da FIA
             '#2b2b2b']  # Preto Pirelli/F1/Fundo padrão


### Entendendo as equipes - histórico de posições

In [7]:
# Unir resultados com nomes das equipes
res_equipe = results.merge(constructors, on='constructorId')

# Contar número de corridas por equipe
corridas_por_equipe = res_equipe['name'].value_counts()
equipes_100_mais = corridas_por_equipe[corridas_por_equipe > 100].index

# Filtrar apenas equipes com mais de 100 corridas
res_filtrado = res_equipe[res_equipe['name'].isin(equipes_100_mais)]

# Plotar boxplot
fig = px.box(
    res_filtrado,
    x='name',
    y='positionOrder',
    title='🏆 Distribuição de posições - Equipes com mais de 100 GPs',
    color_discrete_sequence=['#003399']
)

fig.update_traces(marker_color='#003399')  # Azul da FIA

fig.update_layout(
    xaxis_title='Equipe',
    yaxis_title='Posição na corrida',
    xaxis_tickangle=-45,
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Arial", size=12, color="black"),
    title_font=dict(size=18, color="#003399"),
    yaxis=dict(
        autorange='reversed',  # Inverte eixo y (posição 1 no topo)
        showgrid=True,
        gridcolor='#f0f0f0'
    ),
    xaxis=dict(showgrid=False)
)

# Adicionar imagem do carro no fundo
fig.add_layout_image(
    dict(
        source="https://tse1.mm.bing.net/th/id/OIP.dy2HQjOzt9KYmUF2TnDriQHaHa?cb=thvnextc2&rs=1&pid=ImgDetMain&o=7&rm=3",
        xref="paper",
        yref="paper",
        x=1.00,
        y=1.3,
        sizex=0.2,
        sizey=0.2,
        xanchor="right",
        yanchor="top",
        opacity=0.3,
        layer="below"
    )
)

fig.show()

Este gráfico mostra a distribuição histórica das posições de chegada nas corridas das equipes que ainda estão ativas na Fórmula 1. \
Os principais pontos observados:

- Ferrari, McLaren e Mercedes apresentam as distribuições mais concentradas nas primeiras posições (medianas baixas), o que reflete histórico consistente de bom desempenho.

- Red Bull também apresenta performance sólida, com quartis inferiores baixos e menor dispersão — reflexo de sua dominância nos últimos anos.

- Equipes como Williams e Sauber/Alfa Romeo apresentam grande variabilidade nas posições, indicando histórico com bons e maus momentos.

- Equipes como Haas, AlphaTauri, RB F1 Team e Alpine mostram posições medianas mais altas (próximas de P10–P15), refletindo desempenhos médios ou irregulares ao longo do tempo.

🟪 Interpretação geral: Há uma clara distinção entre as equipes de elite (Mercedes, Ferrari, Red Bull) e as equipes médias/novas. Algumas equipes como Williams tiveram auge no passado mas desempenho decrescente nos últimos anos.

In [8]:
# Pegar IDs de construtores que participaram nos anos mais recentes
races_recentes = races[races['year'] >= 2023]
results_recentes = results[results['raceId'].isin(races_recentes['raceId'])]
ids_atuais = results_recentes['constructorId'].unique()

# Pegar nomes dessas equipes
equipes_atuais = constructors[constructors['constructorId'].isin(ids_atuais)]

# Unir resultados completos com nomes
res_equipe_completo = results.merge(constructors, on='constructorId')

# Filtrar apenas equipes atuais
res_equipes_atuais = res_equipe_completo[res_equipe_completo['constructorId'].isin(ids_atuais)]

# Plotar
fig2 = px.box(
    res_equipes_atuais,
    x='name',
    y='positionOrder',
    title='🏆 Distribuição de posições - Equipes atuais (todos os anos)',
    color_discrete_sequence=['#003399']  # Azul da FIA
)

fig2.update_traces(marker_color='#003399')  # Cor do boxplot

# Atualizar layout
fig2.update_layout(
    xaxis_title='Equipe',
    yaxis_title='Posição na corrida',
    xaxis_tickangle=-45,
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Arial", size=12, color="black"),
    title_font=dict(size=18, color="#003399"),
    yaxis=dict(
        autorange='reversed',  # Menores posições são melhores
        showgrid=True,
        gridcolor='#f0f0f0'
    ),
    xaxis=dict(showgrid=False)
)

# Adicionar imagem do carro
fig2.add_layout_image(
    dict(
        source="https://tse1.mm.bing.net/th/id/OIP.dy2HQjOzt9KYmUF2TnDriQHaHa?cb=thvnextc2&rs=1&pid=ImgDetMain&o=7&rm=3",
        xref="paper", yref="paper",
        x=1.05, y=1.2,
        sizex=0.3, sizey=0.3,
        xanchor="right", yanchor="top",
        opacity=0.3,
        layer="below"
    )
)

fig2.show()

**Entendendo a distribuição de posições entre as equipes com mais de 100 GPs** \
Este gráfico explora a distribuição de posições de todas as equipes com mais de 100 GPs disputados, oferecendo uma visão histórica mais ampla da performance geral.

- McLaren, Ferrari, Williams, Mercedes e Red Bull continuam se destacando com medianas baixas e distribuição mais concentrada entre as primeiras posições.

- Equipes como Arrows, Minardi, March, Osella e Zakspeed apresentam posições medianas altas (P15–P25) e grande dispersão, indicando baixa competitividade ao longo da história.

- Algumas equipes como Jordan, BAR e Jaguar aparecem como equipes intermediárias históricas, com desempenho mediano ao longo do tempo.

🟪 Interpretação geral: Muitas equipes acumularam participações ao longo das décadas sem atingir performance de ponta. Apenas um grupo restrito manteve desempenho competitivo de forma consistente. A dispersão também reflete mudanças de regulamento, entradas/saídas de fabricantes e ciclos de investimento.

### Número de corridas, equipes e pilotos com o passar dos anos

In [9]:
# Juntar races com results para obter o ano e o ID dos pilotos
pilotos_ano = results.merge(races[['raceId', 'year']], on='raceId')
pilotos_por_ano = pilotos_ano.groupby('year')['driverId'].nunique()

In [10]:
# Quantidade de corridas por ano
corridas_por_ano = races['year'].value_counts().sort_index()

# Criar gráfico com Plotly
fig = go.Figure()

fig.add_trace(go.Bar(
    x=corridas_por_ano.index.astype(str),
    y=corridas_por_ano.values,
    marker_color='#003399',  # Azul oficial da FIA
    name='Corridas por ano'
))

fig.update_layout(
    title='🏁 Número de corridas por ano',
    xaxis_title='Ano',
    yaxis_title='Corridas',
    width=1400,
    height=400,
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis=dict(showgrid=False, tickvals=[str(ano) for ano in pilotos_por_ano.index if ano % 5 == 0 or ano == 2024]  # Apenas múltiplos de 5 + 2024
               ),
    yaxis=dict(
        showgrid=True,
        gridcolor='#f0f0f0'
    )
)

# Adicionar imagem do carro
fig.add_layout_image(
    dict(
        source="https://tse1.mm.bing.net/th/id/OIP.dy2HQjOzt9KYmUF2TnDriQHaHa?cb=thvnextc2&rs=1&pid=ImgDetMain&o=7&rm=3",
        xref="paper", yref="paper",
        x=1.00, y=1.35,
        sizex=0.25, sizey=0.25,
        xanchor="right", yanchor="top",
        opacity=0.3,
        layer="below"
    )
)

fig.show()


Houve um aumento no número de corridas de Fórmula 1 desde 1950 e isso está relacionado a vários fatores históricos, econômicos e esportivos:

- **Crescimento e popularidade da Fórmula 1:**\
Desde o início do Campeonato Mundial em 1950, a F1 se tornou cada vez mais popular globalmente, atraindo mais fãs e interesse comercial. Isso estimulou a inclusão de mais corridas no calendário para atender a essa demanda.

- **Expansão geográfica:**\
Inicialmente, as corridas eram concentradas principalmente na Europa. Com o tempo, a F1 expandiu para outros continentes — América do Norte, Ásia, Oriente Médio, Austrália, etc. — o que levou ao aumento no número de Grandes Prêmios anuais.

- **Aspectos comerciais e financeiros:**\
Mais corridas significam maiores receitas para a FIA, equipes, patrocinadores e promotores locais. Países e cidades passaram a investir para receber etapas do campeonato, visando turismo e marketing.

- **Infraestrutura e profissionalização:**\
Com avanços tecnológicos, maior segurança e melhor infraestrutura, tornou-se possível organizar mais eventos durante a temporada com menor risco e maior eficiência.

- **Calendário esportivo:**\
Ao longo das décadas, a F1 passou de calendários curtos (7-9 corridas em média nas primeiras décadas) para calendários atuais com 20, 22 ou até 24 corridas, buscando manter a competição dinâmica e globalizada.

In [11]:
# Juntar races com results para obter o ano e o ID das equipes
equipes_ano = results.merge(races[['raceId', 'year']], on='raceId')

# Contar quantas equipes únicas participaram por ano
equipes_por_ano = equipes_ano.groupby('year')['constructorId'].nunique()

# Criar gráfico com Plotly
fig = go.Figure()

fig.add_trace(go.Bar(
    x=equipes_por_ano.index.astype(str),
    y=equipes_por_ano.values,
    marker_color='#003399',  # Azul da FIA
    name='Equipes por ano'
))

fig.update_layout(
    title='Número de equipes por ano',
    xaxis_title='Ano',
    yaxis_title='Equipes',
    width=1400,
    height=400,
    plot_bgcolor='white',   # Fundo branco
    paper_bgcolor='white',
    xaxis=dict(
        showgrid=False,
        tickvals=[str(ano) for ano in pilotos_por_ano.index if ano % 5 == 0 or ano == 2024]  # Apenas múltiplos de 5 + 2024
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='#f0f0f0'  # Cor clara para o grid horizontal
    )
)

# Adicionar imagem do carro
fig.add_layout_image(
    dict(
        source="https://tse1.mm.bing.net/th/id/OIP.dy2HQjOzt9KYmUF2TnDriQHaHa?cb=thvnextc2&rs=1&pid=ImgDetMain&o=7&rm=3",
        xref="paper", yref="paper",
        x=1.00, y=1.35,
        sizex=0.25, sizey=0.25,
        xanchor="right", yanchor="top",
        opacity=0.3,
        layer="below"
    )
)

fig.show()



In [12]:
# Criar gráfico com Plotly
fig = go.Figure()

fig.add_trace(go.Bar(
    x=pilotos_por_ano.index.astype(str),
    y=pilotos_por_ano.values,
    marker_color='#00d2be',  # Verde água Mercedes
    name='Pilotos por ano'
))

fig.update_layout(
    title='Número de pilotos por ano',
    xaxis_title='Ano',
    yaxis_title='Pilotos',
    width=1400,
    height=400,
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis=dict(
        showgrid=False,
        tickvals=[str(ano) for ano in pilotos_por_ano.index if ano % 5 == 0 or ano == 2024]  # Apenas múltiplos de 5 + 2024
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='#f0f0f0'
    )
)

# Adicionar imagem (maior e com transparência)
fig.add_layout_image(
    dict(
        source="https://static.vecteezy.com/system/resources/previews/036/562/778/large_2x/racing-helmet-icon-vector.jpg",
        xref="paper", yref="paper",
        x=1.00, y=1.35,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="top",
        opacity=0.3,
        layer="below"
    )
)

fig.show()

**Por quê o número de equipes e pilotos participantes diminuiu na F1?**\

🏁 1950–1970: Era “aberta” e acessível\
- Nos anos iniciais da F1 (décadas de 1950 e 60), era comum equipes privadas (“garagistas”) entrarem nas corridas com carros de outras fabricantes.

- O regulamento era mais simples, e o custo para competir era muito menor.

- Alguns anos chegaram a ter 20 ou mais equipes inscritas, mas muitas corriam apenas em 1 ou 2 GPs por temporada.

- Além disso, havia "entries" individuais, sem construtores dedicados.


💸 1980–2000: Profissionalização e aumento dos custos
- A partir dos anos 80 e 90, os carros começaram a exigir desenvolvimento técnico mais avançado, com investimentos em túnel de vento, aerodinâmica, eletrônica e engenharia.

- Com isso, muitas pequenas equipes não conseguiam sustentar os custos crescentes.

- Exemplos de equipes que faliram ou saíram da F1: Minardi, Jordan, Arrows, Prost, Ligier, Lotus (original), entre outras.

- A F1 se tornou um esporte de alto custo e alta tecnologia, com forte dependência de patrocinadores.



🔐 2000–2024: Fechamento do grid e restrições regulatórias
- A FIA passou a limitar o número de equipes no grid, por motivos de segurança e organização (geralmente 10 a 12 equipes).

- Cada equipe agora precisa construir seu próprio chassi — não é mais possível comprar um carro de outro time.

- A entrada na F1 exige aprovação da FIA, pagamento de taxas milionárias e comprovação de estrutura técnica e financeira.

- Exemplo: a Andretti, equipe americana, teve a entrada recusada em 2024, mesmo com estrutura forte, por questões comerciais e políticas.



**Resumo**


| Período   | Nº médio de equipes      | Motivos da variação                         |
| --------- | ------------------------ | ------------------------------------------- |
| 1950–1960 | 15–25 (muitas não fixas) | Acesso fácil, baixo custo                   |
| 1970–1980 | \~15                     | Primeira fase de profissionalização         |
| 1990–2000 | 10–14                    | Alta tecnologia, custo alto                 |
| 2010–2024 | 10 (estável)             | Grid fechado, exigência técnica e comercial |


### Trajetória dos pilotos, equipes e circuitos em número de corridas

In [13]:
# Juntar dados
df = results.merge(races[['raceId', 'year']], on='raceId')
df = df.merge(drivers[['driverId', 'surname']], on='driverId')
df = df.merge(constructors[['constructorId', 'name']], on='constructorId')

# Contar corridas por piloto e ano
corrida_counts = df.groupby(['year', 'driverId', 'surname']).size().reset_index(name='races')
corrida_counts['cumulative_races'] = corrida_counts.groupby('driverId')['races'].cumsum()

# Top 15 por total de corridas
top_15 = corrida_counts.groupby('driverId')['races'].sum().sort_values(ascending=False).head(15).index
corrida_counts = corrida_counts[corrida_counts['driverId'].isin(top_15)]

# Piloto principal na equipe
principal_team = df.groupby(['driverId', 'name']).size().reset_index(name='count')
principal_team = principal_team.sort_values(['driverId', 'count'], ascending=[True, False]).drop_duplicates('driverId')
principal_team = principal_team.set_index('driverId')['name'].to_dict()

# Cores por equipe
team_colors = {
    'Ferrari': '#e10600', 'McLaren': '#ff8700', 'Red Bull': '#1e41ff',
    'Mercedes': '#00d2be', 'Williams': '#005aff', 'Renault': '#f7df1e',
    'Alpine F1 Team': '#0090ff', 'Benetton': '#00b2a9', 'Brawn': '#fff600',
    'AlphaTauri': '#2b4562', 'Aston Martin': '#006f62', 'Jordan': '#ffef00',
    'Lotus': '#00ff00', 'Sauber': '#ff87ab', 'Tyrrell': '#cccccc'
}

# Gráfico
fig = go.Figure()

for driver_id in corrida_counts['driverId'].unique():
    df_pilot = corrida_counts[corrida_counts['driverId'] == driver_id]
    name = df_pilot['surname'].iloc[0]
    team = principal_team.get(driver_id, 'Outros')
    color = team_colors.get(team, 'gray')

    fig.add_trace(go.Scatter(
        x=df_pilot['year'], y=df_pilot['cumulative_races'],
        mode='lines+markers', name=name,
        line=dict(color=color)
    ))

fig.update_layout(
    title='Corridas Acumuladas por Piloto (Top 15)',
    xaxis_title='Ano', yaxis_title='Corridas Acumuladas',
    width=1400, height=500,
    plot_bgcolor='white', paper_bgcolor='white',
    xaxis=dict(
        showgrid=False,
        tickmode='array',
        tickvals=list(range(1950, 2025, 5)) + [2024]
    ),
    yaxis=dict(showgrid=True, gridcolor='#f0f0f0'),
    font=dict(family='Arial', size=12)
)


fig.add_layout_image(
    dict(
        source="https://static.vecteezy.com/system/resources/previews/036/562/778/large_2x/racing-helmet-icon-vector.jpg",
        xref="paper", yref="paper",
        x=1.10, y=1.25,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="top",
        opacity=0.3,
        layer="below"
    )
)

fig.show()

In [14]:
# Dicionário de cores por equipe
cores_equipes = {
    'Ferrari': '#e10600',
    'Red Bull': '#1e41ff',
    'Mercedes': '#00d2be',
    'McLaren': '#ff8700',
    'Williams': '#00A0DE',
    'Renault': '#fff500',
    'Benetton': '#009d9d',
    'Brawn': '#ffffff',
    'Tyrell': '#002147',
    'Ligier': '#4C66FF',
    'BMW Sauber': '#003366'
}

# Juntar resultados com corridas e nomes das equipes
corridas_completo = results.merge(races[['raceId', 'year']], on='raceId')
corridas_completo = corridas_completo.merge(constructors[['constructorId', 'name']], on='constructorId')

# Contar número de corridas por equipe por ano
corridas_ano = corridas_completo.groupby(['year', 'constructorId']).size().reset_index(name='races')

# Total de corridas por equipe
total_corridas = corridas_ano.groupby('constructorId')['races'].sum().reset_index(name='total_races')

# Top 15 equipes com mais corridas
top15_corridas = total_corridas.sort_values(by='total_races', ascending=False).head(15)['constructorId']

# Filtrar apenas essas equipes
corridas_top15 = corridas_ano[corridas_ano['constructorId'].isin(top15_corridas)]

# Adicionar nome da equipe
corridas_top15 = corridas_top15.merge(constructors[['constructorId', 'name']], on='constructorId')

# Corridas acumuladas ao longo do tempo
corridas_top15['cumulative_races'] = corridas_top15.groupby('constructorId')['races'].cumsum()

# Gráfico
fig = go.Figure()

for equipe in corridas_top15['name'].unique():
    equipe_data = corridas_top15[corridas_top15['name'] == equipe]
    cor = cores_equipes.get(equipe, '#888888')  # cinza padrão se não tiver cor definida

    fig.add_trace(go.Scatter(
        x=equipe_data['year'],
        y=equipe_data['cumulative_races'],
        mode='lines+markers',
        name=equipe,
        line=dict(color=cor)
    ))

fig.update_layout(
    title='Corridas Acumuladas ao Longo do Tempo - Top 15 Equipes',
    xaxis_title='Ano',
    yaxis_title='Corridas Acumuladas',
    width=1400,
    height=550,
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Arial", size=12, color="black"),
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=True, gridcolor='#f0f0f0')
)

# Imagem de carro no fundo
fig.add_layout_image(
    dict(
        source="https://tse1.mm.bing.net/th/id/OIP.dy2HQjOzt9KYmUF2TnDriQHaHa?cb=thvnextc2&rs=1&pid=ImgDetMain&o=7&rm=3",
        xref="paper", yref="paper",
        x=1.10, y=1.25,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="top",
        opacity=0.3,
        layer="below"
    )
)

fig.show()

In [15]:
# Juntar 'races' com 'circuits' para ter nome dos circuitos
races_circuitos = races.merge(circuits[['circuitId', 'name']], on='circuitId')

In [16]:
# No groupby e filtros, usar 'name_y' em vez de 'name'
corridas_por_circuito_ano = races_circuitos.groupby(['name_y', 'year']).size().reset_index(name='races_count')

# Ordenar e calcula acumulado
corridas_por_circuito_ano = corridas_por_circuito_ano.sort_values(['name_y', 'year'])
corridas_por_circuito_ano['cumulative_races'] = corridas_por_circuito_ano.groupby('name_y')['races_count'].cumsum()

# Filtrar top 15 circuitos com mais corridas
top_circuitos = corridas_por_circuito_ano.groupby('name_y')['races_count'].sum().sort_values(ascending=False).head(15).index
dados_filtrados = corridas_por_circuito_ano[corridas_por_circuito_ano['name_y'].isin(top_circuitos)]

# Plotar
import plotly.graph_objects as go

fig = go.Figure()

for circuito, df_circuito in dados_filtrados.groupby('name_y'):
    fig.add_trace(go.Scatter(
        x=df_circuito['year'],
        y=df_circuito['cumulative_races'],
        mode='lines+markers',
        name=circuito
    ))

fig.update_layout(
    title='Corridas Acumuladas por Circuito ao Longo dos Anos (Top 15)',
    xaxis_title='Ano',
    yaxis_title='Corridas Acumuladas',
    width=1400,
    height=600,
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family='Arial', size=12),
    xaxis=dict(
        showgrid=False,
        tickmode='array',
        tickvals=list(range(1950, 2025, 5)) + [2024]
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='#f0f0f0'
    )
)

fig.show()

### Trajetória dos pilotos e equipes em número de vitórias

In [17]:
# Top pilotos por vitórias
vitorias = results[results['positionOrder'] == 1].groupby('driverId').size().sort_values(ascending=False)
melhores_pilotos = drivers.set_index('driverId').loc[vitorias.index[:10]]
melhores_pilotos['vitórias'] = vitorias[:10].values

# Dados para o gráfico
nomes = melhores_pilotos['surname']
valores = melhores_pilotos['vitórias']

fig = go.Figure()

fig.add_trace(go.Bar(
    x=nomes,
    y=valores,
    marker_color='#00d2be',  # Verde água Mercedes
    name='Vitórias'
))

fig.update_layout(
    title='🏆 Top 10 Pilotos por Vitórias (número total de vitórias)',
    xaxis_title='Piloto',
    yaxis_title='Vitórias',
    width=1000,
    height=500,
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Arial", size=12, color="black"),
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=True, gridcolor='#f0f0f0')
)

# Adicionar imagem do capacete no fundo
fig.add_layout_image(
    dict(
        source="https://static.vecteezy.com/system/resources/previews/036/562/778/large_2x/racing-helmet-icon-vector.jpg",
        xref="paper", yref="paper",
        x=1.0, y=1.2,
        sizex=0.1, sizey=0.1,
        xanchor="right", yanchor="top",
        opacity=0.3,
        layer="below"
    )
)

fig.show()


In [18]:
# Filtrar vitórias (posição 1)
vitorias = results[results['positionOrder'] == 1].groupby('driverId').size()

# Total de corridas por piloto
corridas = results.groupby('driverId').size()

# Calcular taxa de vitórias
taxa_vitorias = (vitorias / corridas).sort_values(ascending=False)

# Juntar com os dados dos pilotos
top_10_taxa = taxa_vitorias[:10]
pilotos_top_taxa = drivers.set_index('driverId').loc[top_10_taxa.index]
pilotos_top_taxa['Taxa de Vitórias (%)'] = (top_10_taxa.values * 100).round(2)

# Dados para o gráfico
nomes = pilotos_top_taxa['surname']
valores = pilotos_top_taxa['Taxa de Vitórias (%)']

fig = go.Figure()

fig.add_trace(go.Bar(
    x=nomes,
    y=valores,
    marker_color='#00d2be',  # Verde água Mercedes
    name='Taxa de Vitórias (%)'
))

fig.update_layout(
    title='🏆 Top 10 Pilotos com Maior Taxa de Vitórias (%)',
    xaxis_title='Piloto',
    yaxis_title='Taxa de Vitórias (%)',
    width=1000,
    height=500,
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Arial", size=12, color="black"),
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=True, gridcolor='#f0f0f0')
)

fig.add_layout_image(
    dict(
        source="https://static.vecteezy.com/system/resources/previews/036/562/778/large_2x/racing-helmet-icon-vector.jpg",
        xref="paper", yref="paper",
        x=1.0, y=1.2,
        sizex=0.1, sizey=0.1,
        xanchor="right", yanchor="top",
        opacity=0.3,
        layer="below"
    )
)

fig.show()

🏆 Top 10 Pilotos com Maior Taxa de Vitórias (em relação ao total de GPs disputados) \

| Nome                   | Época                | Principais Equipes                      | Comentário                                                                                                                                                |
| ---------------------- | -------------------- | --------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Lee Wallard**        | Início dos anos 1950 | Kurtis Kraft (Indianápolis 500)         | **Participou de apenas 2 GPs** válidos pela F1, venceu 1 — a Indy 500 de 1951. Por isso, a **alta taxa de vitórias**.                                     |
| **Juan Manuel Fangio** | 1950–1958            | Alfa Romeo, Mercedes, Maserati, Ferrari | **5 títulos mundiais**. Época de domínio com várias equipes. Alta taxa de vitórias devido ao talento e à superioridade técnica de suas equipes.           |
| **Bill Vukovich**      | Anos 1950            | Kurtis Kraft (Indy 500)                 | Como Wallard, correu apenas **em Indianápolis**, válida pela F1. Venceu 2 das 5 corridas.                                                                 |
| **Alberto Ascari**     | 1950–1955            | Ferrari, Lancia                         | **Bicampeão mundial (1952, 1953)**. Muito dominante com a Ferrari na era pré-Fangio.                                                                      |
| **Jim Clark**          | 1960–1968            | Lotus                                   | Considerado um dos **maiores pilotos de todos os tempos**. Bicampeão (1963, 1965), com grande eficiência, mas teve carreira interrompida tragicamente.    |
| **Max Verstappen**     | 2015–presente        | Red Bull Racing                         | Dominante desde 2021, especialmente 2022–2024. Sua taxa de vitórias subiu drasticamente com o **carro dominante da Red Bull**.                            |
| **Michael Schumacher** | 1991–2012            | Benetton, Ferrari                       | **7 títulos mundiais**. Período de grande domínio na Ferrari entre 2000–2004. Teve também muitas corridas fora da era dominante.                          |
| **Lewis Hamilton**     | 2007–presente        | McLaren, Mercedes                       | **7 títulos mundiais**, maior número de vitórias totais. Sua taxa é um pouco diluída pelo início de carreira e temporadas mais equilibradas recentemente. |
| **Jackie Stewart**     | 1965–1973            | BRM, Matra, Tyrrell                     | **Tricampeão mundial**. Extremamente eficiente e regular, pilotou antes da era da ultra-dominância.                                                       |
| **Ayrton Senna**       | 1984–1994            | Toleman, Lotus, McLaren, Williams       | **Tricampeão**, lenda da F1. Alta taxa mesmo enfrentando forte concorrência e carros nem sempre dominantes.                                               |



Comentários gerais:
- Os três primeiros (Wallard, Vukovich e Fangio) beneficiam-se de contexto histórico específico: poucas corridas, menos pilotos e equipes menos homogêneas.

- Pilotos como Clark, Ascari e Stewart mostram como mesmo em épocas menos tecnológicas, a eficiência individual era marcante.

- Verstappen aparece entre os maiores em eficiência moderna, reflexo do domínio atual da Red Bull.

- Hamilton e Schumacher, apesar de serem os maiores em números absolutos, não estão no topo da taxa porque disputaram muitos GPs fora do auge.

- Senna, mesmo com temporadas difíceis e algumas quebras, manteve um índice altíssimo.

In [19]:
# Filtrar apenas vitórias
victories = results[results['positionOrder'] == 1]

# Juntar com dados de corrida e piloto
victories = victories.merge(races[['raceId', 'year']], on='raceId')
victories = victories.merge(drivers[['driverId', 'surname']], on='driverId')
victories = victories.merge(constructors[['constructorId', 'name']], on='constructorId')

# Determinar o piloto principal por tempo na equipe
principal_team = victories.groupby(['driverId', 'name']).size().reset_index(name='count')
principal_team = principal_team.sort_values(['driverId', 'count'], ascending=[True, False]).drop_duplicates('driverId')
principal_team = principal_team.set_index('driverId')['name'].to_dict()

# Contar vitórias por ano e piloto
victory_counts = victories.groupby(['year', 'driverId', 'surname']).size().reset_index(name='wins')
victory_counts['cumulative_wins'] = victory_counts.groupby('driverId')['wins'].cumsum()

# Top 15 por total de vitórias
top_15 = victory_counts.groupby('driverId')['wins'].sum().sort_values(ascending=False).head(15).index
victory_counts = victory_counts[victory_counts['driverId'].isin(top_15)]

# Cores por equipe
team_colors = {
    'Ferrari': '#e10600', 'McLaren': '#ff8700', 'Red Bull': '#1e41ff',
    'Mercedes': '#00d2be', 'Williams': '#005aff', 'Renault': '#f7df1e',
    'Alpine F1 Team': '#0090ff', 'Benetton': '#00b2a9', 'Brawn': '#fff600',
    'AlphaTauri': '#2b4562', 'Aston Martin': '#006f62', 'Jordan': '#ffef00',
    'Lotus': '#00ff00', 'Sauber': '#ff87ab', 'Tyrrell': '#cccccc'
}

# Criar gráfico
fig = go.Figure()

for driver_id in victory_counts['driverId'].unique():
    df = victory_counts[victory_counts['driverId'] == driver_id]
    name = df['surname'].iloc[0]
    team = principal_team.get(driver_id, 'Outros')
    color = team_colors.get(team, 'gray')

    fig.add_trace(go.Scatter(
        x=df['year'], y=df['cumulative_wins'],
        mode='lines+markers', name=name,
        line=dict(color=color)
    ))

fig.update_layout(
    title='Vitórias Acumuladas por Piloto (Top 15)',
    xaxis_title='Ano', yaxis_title='Vitórias Acumuladas',
    width=1400, height=500,
    plot_bgcolor='white', paper_bgcolor='white',
    xaxis=dict(
        showgrid=False,
        tickmode='array',
        tickvals=list(range(1950, 2025, 5)) + [2024]
    ),
    yaxis=dict(showgrid=True, gridcolor='#f0f0f0'),
    font=dict(family='Arial', size=12)
)


fig.add_layout_image(
    dict(
        source="https://static.vecteezy.com/system/resources/previews/036/562/778/large_2x/racing-helmet-icon-vector.jpg",
        xref="paper", yref="paper",
        x=1.10, y=1.25,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="top",
        opacity=0.3,
        layer="below"
    )
)

fig.show()

In [20]:
# Processar top_decada
res_com_ano = results.merge(races[['raceId', 'year']], on='raceId')
res_com_ano['década'] = (res_com_ano['year'] // 10) * 10
vitorias_decada = res_com_ano[res_com_ano['positionOrder'] == 1].groupby(['década', 'driverId']).size().reset_index(name='vitórias')
top_decada = vitorias_decada.sort_values(['década','vitórias'], ascending=[True, False]).groupby('década').head(1)
top_decada = top_decada.merge(drivers, on='driverId')

# Adicionar a equipe principal e cor
top_decada['team'] = top_decada['driverId'].map(principal_team)
top_decada['color'] = top_decada['team'].map(team_colors).fillna('gray')

# Criar o mapa de cores por sobrenome
color_map = {row['surname']: row['color'] for _, row in top_decada.iterrows()}

# Gerar gráfico principal
fig = px.bar(
    top_decada,
    x='década',
    y='vitórias',
    text='surname',
    title='Piloto com Mais Vitórias por Década',
    labels={'vitórias': 'Número de Vitórias', 'década': 'Década'},
    color='surname',
    color_discrete_map=color_map
)

# Ocultar os nomes dos pilotos na legenda
fig.update_traces(
    textposition='outside',
    showlegend=False  # <- Aqui desativa a legenda dos pilotos
)

# Adicionar traços invisíveis para criar a legenda com base nas equipes
equipes_usadas = top_decada['team'].dropna().unique()
for equipe in sorted(equipes_usadas):
    cor = team_colors.get(equipe, 'gray')
    fig.add_trace(go.Scatter(
        x=[None], y=[None],  # ponto invisível
        mode='markers',
        marker=dict(size=10, color=cor),
        legendgroup=equipe,
        showlegend=True,
        name=equipe
    ))

# Atualizar layout
fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family='Arial', size=12),
    xaxis=dict(
        tickmode='linear',
        dtick=10,
        showgrid=False
    ),
    yaxis=dict(
        gridcolor='#f0f0f0',
        title='Número de Vitórias'
    ),
    showlegend=True,
    legend_title_text='Equipe Principal'
)

fig.show()

In [21]:
# Cores definidas manualmente por equipe
cores_equipes = {
    'Ferrari': '#e10600',
    'Red Bull': '#1e41ff',
    'Mercedes': '#00d2be',
    'McLaren': '#ff8700',
    'Williams': '#00A0DE',
    'Renault': '#fff500',
    'Benetton': '#009d9d',
    'Brawn': '#ffffff',
    'Tyrell': '#002147',
    'Ligier': '#4C66FF',
    'BMW Sauber': '#003366'
}

# Filtrar apenas vitórias
victories = results[results['positionOrder'] == 1]

# Juntar com corridas para pegar ano
victories = victories.merge(races[['raceId', 'year']], on='raceId')

# Contar vitórias por ano e equipe
victories_count = victories.groupby(['year', 'constructorId']).size().reset_index(name='wins')

# Somar total de vitórias por equipe
total_wins = victories_count.groupby('constructorId')['wins'].sum().reset_index(name='total_wins')

# Pegar os 15 com mais vitórias
top_15_ids = total_wins.sort_values(by='total_wins', ascending=False).head(15)['constructorId']

# Filtrar apenas essas equipes
victories_count = victories_count[victories_count['constructorId'].isin(top_15_ids)]

# Adicionar nomes das equipes
victories_count = victories_count.merge(constructors[['constructorId', 'name']], on='constructorId')

# Calcular vitórias acumuladas
victories_count['cumulative_wins'] = victories_count.groupby('constructorId')['wins'].cumsum()

# Gráfico
fig = go.Figure()

for constructor in victories_count['name'].unique():
    constructor_data = victories_count[victories_count['name'] == constructor]
    cor = cores_equipes.get(constructor, '#888888')  # cinza padrão se não definida

    fig.add_trace(go.Scatter(
        x=constructor_data['year'],
        y=constructor_data['cumulative_wins'],
        mode='lines+markers',
        name=constructor,
        line=dict(color=cor)
    ))

fig.update_layout(
    title='🏁 Vitórias Acumuladas ao Longo do Tempo - Top 15 Equipes',
    xaxis_title='Ano',
    yaxis_title='Vitórias Acumuladas',
    width=1400,
    height=550,
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Arial", size=12, color="black"),
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=True, gridcolor='#f0f0f0')
)

# Imagem de carro
fig.add_layout_image(
    dict(
        source="https://tse1.mm.bing.net/th/id/OIP.dy2HQjOzt9KYmUF2TnDriQHaHa?cb=thvnextc2&rs=1&pid=ImgDetMain&o=7&rm=3",
        xref="paper", yref="paper",
        x=1.1, y=1.25,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="top",
        opacity=0.3,
        layer="below"
    )
)

fig.show()

In [22]:
# Equipes com mais títulos de construtores
constr_titulos = constructor_standings[constructor_standings['position'] == 1].groupby('constructorId').size()
construtores = constructors.set_index('constructorId').loc[constr_titulos.index[:10]]
construtores['títulos'] = constr_titulos[:10].values

# Ordenar decrescente pelo número de títulos
construtores = construtores.sort_values(by='títulos', ascending=False)

# Dados para o gráfico
nomes = construtores['name']
valores = construtores['títulos']

fig = go.Figure()

fig.add_trace(go.Bar(
    x=nomes,
    y=valores,
    marker_color='#003399',  # Azul FIA
    name='Títulos'
))

fig.update_layout(
    title='Top Equipes Campeãs',
    xaxis_title='Equipe',
    yaxis_title='Títulos',
    width=1000,
    height=500,
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Arial", size=12, color="black"),
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=True, gridcolor='#f0f0f0')
)

fig.add_layout_image(
    dict(
        source="https://tse1.mm.bing.net/th/id/OIP.dy2HQjOzt9KYmUF2TnDriQHaHa?cb=thvnextc2&rs=1&pid=ImgDetMain&o=7&rm=3",
        xref="paper", yref="paper",
        x=1.00, y=1.2,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="top",
        opacity=0.3,
        layer="below"
    )
)

fig.show()

A soberania das 4 grandes equipes: FErrari, Mclaren, Red Bull e Williams, se dá por:

🟥 Ferrari (234 vitórias)
- Maior vencedora da história da F1.

- Participa desde 1950, a única equipe presente em todas as temporadas.

- Teve várias fases de ouro, como:

- 1950–1960: Era Juan Manuel Fangio e Alberto Ascari.

- 1970s: Com Niki Lauda e Jody Scheckter.

- 2000–2004: Domínio absoluto com Michael Schumacher (5 títulos seguidos).

- Longa permanência e recursos da montadora explicam a constância de vitórias.

\

🟧 McLaren (173 vitórias)
- Fundada nos anos 60 por Bruce McLaren.

- Fases dominantes:

  - 1980s: Era de Ayrton Senna, Alain Prost, motores Honda.

  - 1998–1999: Títulos com Mika Häkkinen e motor Mercedes.

- Forte parceria com grandes pilotos e motores eficientes em vários períodos.

\

🔵 Red Bull (124 vitórias)
- Uma das mais recentes dominantes.

- Entrou na F1 como equipe em 2005.

- Domínios marcantes:

  - 2010–2013: 4 títulos seguidos com Sebastian Vettel.

  - 2021–2024 (atual): Era Max Verstappen, domínio quase absoluto.

- Crescimento rápido graças ao investimento da marca e engenharia de Adrian Newey.

\

🔷 Williams (114 vitórias)
- Fundada por Frank Williams.

- Teve seu auge entre:

  - 1980s–1990s: Com Nelson Piquet, Nigel Mansell, Alain Prost, Damon Hill, e motores Honda/Renault.

- Era referência técnica até o final dos anos 90.

- Entrou em declínio nos anos 2000, mas ainda carrega um legado muito forte.

### Voltas Mais Rápidas

Esta análise mostra os pilotos com maior número de voltas mais rápidas com o passar do tempo.

In [23]:
# adicionar coluna 'year' em results via merge com races
results_year = results.merge(races[['raceId', 'year']], on='raceId')

# filtrar por voltas que não são nulas e pega o índice da volta mais rápida por corrida
idx_fastest_laps = results_year.groupby('raceId')['fastestLapTime'].idxmin()

# filtrar apenas as voltas mais rápidas por corrida
fastest_laps = results_year.loc[idx_fastest_laps]

# contar número de voltas mais rápidas por piloto por ano
fastest_laps_count = fastest_laps.groupby(['driverId', 'year']).size().reset_index(name='fastest_laps')

# calcular cumulativo por piloto ao longo dos anos
fastest_laps_count['cumulative_fastest_laps'] = fastest_laps_count.groupby('driverId')['fastest_laps'].cumsum()

# juntar com nomes dos pilotos
fastest_laps_count = fastest_laps_count.merge(drivers[['driverId', 'surname']], on='driverId')

# filtrar os pilotos que tiveram pelo menos uma volta mais rápida (top 15)
top_15_drivers = fastest_laps_count.groupby('driverId')['fastest_laps'].sum().sort_values(ascending=False).head(15).index
fastest_laps_count = fastest_laps_count[fastest_laps_count['driverId'].isin(top_15_drivers)]

# achar a equipe principal de cada piloto
driver_teams = (
    results.merge(constructors, on='constructorId')
           .groupby(['driverId', 'name']).size()
           .reset_index(name='count')
           .sort_values(['driverId', 'count'], ascending=[True, False])
           .drop_duplicates('driverId')
           .set_index('driverId')['name']
)

# paleta de cores das equipes
team_colors = {
    'Ferrari': '#e10600',
    'McLaren': '#ff8700',
    'Red Bull': '#1e41ff',
    'Mercedes': '#00d2be',
    'Renault': '#f7df1e',
    'Williams': '#005aff',
    'AlphaTauri': '#2b4562',
    'Alpine F1 Team': '#0090ff',
    'Aston Martin': '#006f62',
    'Sauber': '#51c8ff',
    'BMW Sauber': '#0038a8',
    'Benetton': '#48c0a2',
    'Lotus F1': '#555d50',
    'Brawn': '#d5ff00',
    'Jordan': '#f4d30f',
    'Ligier': '#1c39bb',
    'Tyrrell': '#cccccc'
}

# criar gráfico plotly
fig = go.Figure()

for driver_id, df_pilot in fastest_laps_count.groupby('driverId'):
    nome = df_pilot['surname'].iloc[0]
    equipe = driver_teams.get(driver_id, 'Outros')
    cor = team_colors.get(equipe, 'gray')

    fig.add_trace(go.Scatter(
        x=df_pilot['year'],
        y=df_pilot['cumulative_fastest_laps'],
        mode='lines+markers',
        name=nome,
        line=dict(color=cor)
    ))

fig.update_layout(
    title='Voltas Mais Rápidas Acumuladas por Piloto (Top 15)',
    xaxis_title='Ano',
    yaxis_title='Voltas Mais Rápidas Acumuladas',
    width=1400,
    height=550,
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family='Arial', size=13),
    xaxis=dict(
        showgrid=False,
        tickmode='array',
        tickvals=list(range(1950, 2025, 5)) + [2024]
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='#f0f0f0'
    )
)

fig.add_layout_image(
    dict(
        source="https://tse1.mm.bing.net/th/id/OIP.dy2HQjOzt9KYmUF2TnDriQHaHa?cb=thvnextc2&rs=1&pid=ImgDetMain&o=7&rm=3",
        xref="paper", yref="paper",
        x=1.10, y=1.2,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="top",
        opacity=0.3,
        layer="below"
    )
)

fig.show()

In [24]:
# Função para converter strings de tempo (ex: "1:26.572", "1:34:50.616", "+5.478") em segundos
def convert_time_to_sec(time_str):
    if pd.isna(time_str) or time_str == "" or time_str == "\\N":
        return np.nan
    time_str = time_str.lstrip('+')  # remove sinais de +
    parts = time_str.split(':')
    if len(parts) == 2:
        minutes = float(parts[0])
        seconds = float(parts[1])
        return minutes * 60 + seconds
    elif len(parts) == 3:
        hours = float(parts[0])
        minutes = float(parts[1])
        seconds = float(parts[2])
        return hours * 3600 + minutes * 60 + seconds
    else:
        return np.nan

# Filtrar circuitos Monza e Monaco
circuitos_alvo = circuits[circuits['name'].isin(['Autodromo Nazionale di Monza', 'Circuit de Monaco'])]
circuitos_ids = circuitos_alvo.set_index('circuitId')['name'].to_dict()

# Prepara lap_times: converte tempo das voltas de milissegundos para segundos (coluna 'lap_sec')
lap_times['lap_sec'] = lap_times['milliseconds'] / 1000

# Juntar lap_times com races para obter year e circuitId
lap_times_races = lap_times.merge(races[['raceId', 'year', 'circuitId']], on='raceId')

# Filtrar apenas os circuitos desejados e o período de 2005 a 2024
lap_times_filtrado = lap_times_races[
    (lap_times_races['circuitId'].isin(circuitos_ids.keys())) &
    (lap_times_races['year'] >= 2005) & (lap_times_races['year'] <= 2024)
]

# Calcular o tempo médio por ano e circuito
tempo_medio_ano_circuito = lap_times_filtrado.groupby(['year', 'circuitId'])['lap_sec'].mean().reset_index()

# Mapeia circuitId para nome do circuito
tempo_medio_ano_circuito['circuitName'] = tempo_medio_ano_circuito['circuitId'].map(circuitos_ids)

# Anos para as linhas verticais tracejadas
event_years = [2010, 2011, 2014, 2016, 2019, 2021, 2022]

# Criar shapes para cada linha vertical
vertical_lines = []
for year in event_years:
    vertical_lines.append(
        dict(
            type="line",
            xref="x",
            yref="paper",
            x0=year,
            y0=0,
            x1=year,
            y1=1,
            line=dict(
                color="gray",
                width=1,
                dash="dashdot"
            )
        )
    )

# Plot com Plotly Express
fig = px.line(
    tempo_medio_ano_circuito,
    x='year',
    y='lap_sec',
    color='circuitName',
    title='Tempo Médio das Voltas (2005-2024) - Monza vs Monaco (lap_times)',
    labels={'lap_sec': 'Tempo Médio da Volta (s)', 'year': 'Ano', 'circuitName': 'Circuito'}
)

fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis=dict(
        showgrid=False,
        tickmode='array',
        tickvals=list(range(2005, 2025, 5)) + [2024]
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='#f0f0f0'
    ),
    font=dict(family='Arial', size=13),
    shapes=vertical_lines  # adiciona as linhas verticais tracejadas
)

# Mapeia eventos por ano
eventos = {
    2010: "2010: Reabastecimento",
    2011: "2011: DRS + KERS",
    2014: "2014: Motores híbridos V6 turbo",
    2016: "2016: Limite mensagens de rádio",
    2019: "2019: Ponto extra por volta mais rápida",
    2021: "2021: Redução downforce",
    2022: "2022: Efeito solo"
}

# Adiciona anotações no topo para cada evento (em 90º)
for year, text in eventos.items():
    fig.add_annotation(
        x=year - 0.12,
        y=1.02,  # levemente acima do gráfico
        xref="x",
        yref="paper",
        text=text,
        showarrow=False,
        font=dict(size=11, color="gray"),
        align="center",
        textangle=270  # gira o texto para ficar na vertical
    )

# Layout e exibição
fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis=dict(
        showgrid=False,
        tickmode='array',
        tickvals=list(range(2005, 2025, 5)) + [2024]
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='#f0f0f0'
    ),
    font=dict(family='Arial', size=13),
    shapes=vertical_lines,
    margin=dict(t=100)  # aumenta o topo para acomodar as anotações
)

fig.show()

### Linha do tempo - Eventos históricos


| Ano    | Mudança Regulamentar ou Técnica Principal                                                              |
| ------ | ------------------------------------------------------------------------------------------------------ |
| 1968   | Introdução do **capacete integral** e **primeiros patrocínios comerciais**                             |
| 1969   | **Proibição das asas móveis** (ajustáveis em tempo real)                                               |
| 1970   | Regras de **tanques de combustível mais seguros** e regulamentos de **pistas**                         |
| 1983   | **Fim do efeito solo** (uso de assoalhos planos obrigatório)                                           |
| 1994   | Banimento de **ajudas eletrônicas** (controle de tração, ABS, suspensão ativa) + foco em **segurança** |
| 2003   | **Sistema de pontuação modificado** (mais pilotos pontuando: 10-8-6-5-4-3-2-1)                         |
| 2009   | **Reforma aerodinâmica geral**, pneus slicks retornam, introdução do **KERS** opcional                 |
| 2010   | **Proibição de reabastecimento em corridas** (**refuelling ban**), alteração no formato de pit stops   |
| 2011   | Introdução do **DRS (Drag Reduction System)** + KERS se torna mais comum                               |
| 2014   | Introdução dos motores híbridos **V6 turbo (1.6L)** com **ERS**                                        |
| 2016   | **Limite de mensagens de rádio** para evitar "coaching"                                                |
| 2019   | **Ponto extra por volta rápida** se estiver entre os 10 primeiros                                      |
| 2021   | Mudanças no assoalho e difusor para reduzir o downforce                                                |
| 2022   | **Novo regulamento aerodinâmico**: retorno do **efeito solo** com foco em reduzir o ar sujo            |
| 2025\* | Previsão de **revisões técnicas** e **fim do ponto da volta rápida**, foco em segurança e estreantes   |



Olhando o gráfico à luz dos eventos históricos de 2010 a 2022:

✅ 2009 → 2010: Fim do reabastecimento
   - Impacto esperado: Carros mais pesados no início da corrida = voltas mais lentas no começo.
    - Observado no gráfico:
       - Em Mônaco: Pequeno aumento no tempo médio.
      - Em Monza: Leve queda - o que gere curiosidade mas pode ser relacionado ao menor impacto no peso em um circuito de alta velocidade e com poucas curvas lentas.

\

✅ 2010 → 2011: DRS + KERS
  - Impacto esperado: Redução dos tempos de volta, especialmente em retas (circuito de Monza).
  - Observado no gráfico:
    - Monza: Forte aumento no tempo, o que é surpreendente, e pode indicar outros fatores como clima ou safety car.
    - Mônaco: Aumento ainda maior. O DRS e KERS tem menor efeito em circuitos de rua, então não faria sentido uma redução significativa nos tempos de volta, mas um aumento significativo também não se sustenta.

\

✅ 2013 → 2014: Motores híbridos V6 turbo
  - Impacto esperado: Aumento de tempos no início da era, devido à curva de aprendizado com novas unidades de potência.

  - Observado:

    - Ambos os circuitos apresentam redução de tempo médio, o que pode parecer contraintuitivo.

    - Explicação: motores novos + torque instantâneo do ERS pode ter compensado perdas, ou houve melhora em pneus, aerodinâmica, ou redução de safety cars.

\

✅ 2015 → 2016: Limite nas mensagens de rádio
- Impacto esperado: Possível perda de performance por menor coaching.

- Observado:

  - Leve aumento em Monaco, consistente com a exigência técnica do circuito.

  - Monza se mantém estável.

\

✅ 2018 → 2019: Ponto extra por volta rápida
- Impacto esperado: Estímulo a voltas rápidas no fim da corrida.

- Observado:

  - Nenhuma mudança abrupta. Pode não impactar o tempo médio, mas sim apenas a volta mais rápida.

\

✅ 2020 → 2021: Redução do downforce
- Impacto esperado: Carros mais lentos em curvas.

- Observado:

  - Monza: aumento notável de tempo (esperado).

  - Monaco: tempo médio sobe drasticamente (congruente com a perda de aderência em curvas fechadas).

\


✅ 2021 → 2022: Novo regulamento aerodinâmico (efeito solo)
- Impacto esperado: Melhor desempenho em curvas, menos ar sujo.

- Observado:

  - Monaco: queda visível no tempo médio.

  - Monza: queda ainda maior — o efeito solo favorece estabilidade em curvas rápidas como as de Monza.

\

📌 Outros pontos relevantes
Flutuações fortes (ex: Monaco 2011, 2021, 2024) podem estar associadas a:

- Corridas interrompidas por bandeiras vermelhas.

- Alta incidência de safety cars.

- Mudanças climáticas (corrida molhada).

- Poucos dados válidos naquele ano.

## Pré-processamento de Dados

Após a análise exploratória, uma possibilidade seria continuar o trabalho com a aplicação de uma classificação supervisionada para previsão de pilotos vencedores dados alguns atributos.

Para isso, inicialmente deve-se realizar o pré-processamento do dados, verificando valores ausentes, tipos e consistência básica, seguindo com uma padronização.

In [25]:
# Verificando valores ausentes
missing_df = pd.DataFrame({
    'drivers': drivers.isnull().sum(),
    'constructors': constructors.isnull().sum(),
    'races': races.isnull().sum(),
    'results': results.isnull().sum(),
    'driver_standings': driver_standings.isnull().sum(),
    'constructor_standings': constructor_standings.isnull().sum(),
    'lap_times': lap_times.isnull().sum(),
    'qualifying': qualifying.isnull().sum()
})
missing_df

,drivers,constructors,races,results,driver_standings,constructor_standings,lap_times,qualifying
circuitId,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
code,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
constructorId,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
constructorRef,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
constructorStandingsId,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
date,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
dob,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
driverId,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0
driverRef,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
driverStandingsId,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [26]:
# Padronização
features_ml = results[['grid', 'positionOrder']].dropna()
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_ml[['grid']])

## Classificação Supervisionada: Previsão de Vencedores

In [27]:
# Variável alvo: venceu ou não
df_ml = results[['grid', 'positionOrder']].dropna()
df_ml['vencedor'] = (df_ml['positionOrder'] == 1).astype(int)

X = df_ml[['grid']]
y = df_ml['vencedor']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Modelo
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Acurácia:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Acurácia: 0.9569008470353761
[[7682    0]
 [ 346    0]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      7682
           1       0.00      0.00      0.00       346

    accuracy                           0.96      8028
   macro avg       0.48      0.50      0.49      8028
weighted avg       0.92      0.96      0.94      8028



# Conclusão

A análise histórica e exploratória dos dados da Fórmula 1 permitiu observar tendências interessantes na dominância de pilotos e equipes. \
Além disso, foi possível direcionar os dados para uma futura aplicação de um modelo supervisionado, que poderá mostrar que a determinado atributo, como a posição de largada do piloto, possua alguma relevância preditiva sobre a vitória desse piloto, possibilitando uma base simples para modelos futuros.

# Pontos de melhoria

Há a possibilidade de ir mais além nas análises, como realizar uma comparação do tempo médio das voltas por década em diferentes circuitos, a evolução da diferença entre o piloto mais rápido e o mais lento de cada corrida - verificando a trajetória ao longo dos anos em relação à competitividade na F1, etc.